In [1]:
#Podemos extraer el año, el nombre de la película, el numero de votos,la duracion y la calificación de la película de la página web de IMDB.

import requests
from bs4 import BeautifulSoup

def fetch_movies(url, headers):
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    return soup


In [2]:
import csv
#Estas listas las he usado para ir comprobando com ofuncionaba el código, al final habría que eliminarlas
lista_dur=[]
movie_IDBMn=[]
movie_IDBMc=[]
movie_IDBMr=[]
movie_IDBMpack=[]

def extract_movie_data(soup):

    movie_data = []
    #find_all busca todos los elementos que coincidan con los criterios de búsqueda y los guarda en una lista
    movie_names = soup.find_all('h3', class_='ipc-title__text')
    movie_cal = soup.find_all('span', class_='ipc-rating-star ipc-rating-star--base ipc-rating-star--imdb ratingGroup--imdb-rating')
    movie_reviews = soup.find_all('span', class_='ipc-rating-star--voteCount')
    movie_numbers = soup.find_all('div', class_="sc-b0691f29-7 hrgukm cli-title-metadata")
    movie_pack = soup.find_all('div', class_='sc-b0691f29-0 jbYPfh cli-children')

    #Añadimos los datos a las listas
    movie_IDBMn.append(movie_names)
    movie_IDBMc.append(movie_cal)
    movie_IDBMr.append(movie_reviews)
    lista_dur.append(movie_numbers)
    movie_IDBMpack.append(movie_pack)

    # Rest of the code

    for name, cal, reviews, numbers in zip(movie_names[1:], movie_cal, movie_reviews, movie_numbers):
        movie_name = name.text.strip()
        print(movie_name)
        name_index=movie_name.find(" ")

        #Se procesan algunos datos
        if reviews == 0.0:
            movie_reviews = 0.0
        else:
            movie_reviews = reviews.text.strip().replace(')', '').replace('(', '')

        if cal == 0.0:
            movie_cal_str = 0.0
        else:
            movie_cal_str = cal.text.split()[0]

        movie_numbers = numbers.text.strip()
        #print(len(movie_numbers))

        year = movie_numbers[0:4]

        #procesar duracion de la película enfunción de la posicion de m, los minutos
        duration_index = movie_numbers.find("m")
        if duration_index != -1:
            duration = movie_numbers[4:duration_index+1]
            rating = movie_numbers[duration_index+1:].strip()

        #si no hay regsitrados minutos, es decir, si la película dura 1 o 2 horas exactas por ejemplo, no hay m así que
        #se procesa a partir de h y de la posición del "." de la puntuación de la película.
        #Aunque la puntuación de la película sea un numero entero se registra como decimal ("7.0") así que funciona siempre
        else:
            duration_index = movie_numbers.find("h")
            rate_index = movie_numbers.find(".")
            duration = movie_numbers[duration_index-1:rate_index-1]
            rating = movie_numbers[rate_index-1: ]


        try:
            movie_cal = float(movie_cal_str)
            movie_data.append((movie_name[name_index+1:], movie_cal, movie_reviews, year, duration, rating))
        except ValueError:
            print(f"Could not convert rating '{movie_cal_str}' for movie '{movie_name}' to a float.")

    return movie_data,lista_dur

In [3]:
import csv
filename='movies.csv'
def write_movies_to_csv(movies, min_rating, filename):

    # Guardar la información de las películas en un archivo CSV
    #he quitado ratings porque estaba dando problemas
    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['Movie Name', 'Calification',"Number of reviews","Year","Duration","Rating"]#,"Year","Duration","Rating"
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for movie in movies:#filtered_movies
            writer.writerow({'Movie Name': movie[0], 'Calification': movie[1], "Number of reviews": movie[2], "Year": movie[3], "Duration": movie[4], "Rating": movie[5]})#, "Year": movie[3], "Duration": movie[4], "Rating": movie[5]


In [4]:
# Main program
# la url a la que accedemos
#url="https://www.imdb.com/chart/moviemeter/?sort=release_date%2Cdesc"
url="https://www.imdb.com/chart/top/"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

soup = fetch_movies(url, headers)
movies,lista_dur = extract_movie_data(soup)

# El código original lo usaba para comprobar que los datos se habían extraído correctamente
#lo he puesto en 0 para que muestre todas las peliculas
min_rating = 0
write_movies_to_csv(movies, min_rating, filename)

1. The Shawshank Redemption
2. The Godfather
3. The Dark Knight
4. The Godfather Part II
5. 12 Angry Men
6. Schindler's List
7. The Lord of the Rings: The Return of the King
8. Pulp Fiction
9. The Lord of the Rings: The Fellowship of the Ring
10. The Good, the Bad and the Ugly
11. Forrest Gump
12. The Lord of the Rings: The Two Towers
13. Fight Club
14. Inception
15. Dune: Part Two
16. Star Wars: Episode V - The Empire Strikes Back
17. The Matrix
18. Goodfellas
19. One Flew Over the Cuckoo's Nest
20. Se7en
21. Interstellar
22. It's a Wonderful Life
23. Seven Samurai
24. The Silence of the Lambs
25. Saving Private Ryan
26. City of God
27. Life Is Beautiful
28. The Green Mile
29. Terminator 2: Judgment Day
30. Star Wars: Episode IV - A New Hope
31. Back to the Future
32. Spirited Away
33. The Pianist
34. Parasite
35. Psycho
36. Spider-Man: Across the Spider-Verse
37. Gladiator
38. The Lion King
39. Léon: The Professional
40. The Departed
41. American History X
42. Whiplash
43. The Presti

A partir de aquí son celdas en las que hice pruebas etc.

In [ ]:
print("names",len(movie_IDBMn[0]),
      "cal",len(movie_IDBMc[0]),
            "rev",len(movie_IDBMr[0]),
                  "numbers",len(lista_dur[0]))


names 262 cal 250 rev 250 numbers 250


In [ ]:
lista_dur[0][15]

<div class="sc-b0691f29-7 hrgukm cli-title-metadata"><span class="sc-b0691f29-8 ilsLEX cli-title-metadata-item">1980</span><span class="sc-b0691f29-8 ilsLEX cli-title-metadata-item">2h 4m</span><span class="sc-b0691f29-8 ilsLEX cli-title-metadata-item">A</span></div>

In [ ]:
movie_IDBMn[0][197].text.strip().find(" ")

4

In [ ]:
# #contar el número de "Null" en la columna de duracion del csv movies.csv

# import pandas as pd
# df=pd.read_csv('movies.csv')
# print(df['Duration'].value_counts()['Null'])